In [1]:
] add https://github.com/vchuravy/MCAnalyzer.jl

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
  Updating git-repo `https://github.com/vchuravy/MCAnalyzer.jl`
  Updating git-repo `https://github.com/vchuravy/MCAnalyzer.jl`
 Resolving package versions...
  Updating `~/.julia/environments/v1.3/Project.toml`
  [a81df072] ~ MCAnalyzer v0.1.0 #master (https://github.com/vchuravy/MCAnalyzer.jl)
  Updating `~/.julia/environments/v1.3/Manifest.toml`
  [a81df072] ~ MCAnalyzer v0.1.0 #master (https://github.com/vchuravy/MCAnalyzer.jl)


In [1]:
3+4

7

In [2]:
using LLVM
using LLVM.Interop
import MCAnalyzer: irgen

In [29]:
fieldtypes(Holder)

(Float64, String)

In [37]:
eltype(Array{Float64,1})

Float64

In [3]:
const _mod = Ref{LLVM.Module}()

@enum Diffe begin
   Duplicate = 1
   Output = 2
   Constant = 3
end

function hasfieldcount(@nospecialize(dt))
    try
        fieldcount(dt)
    catch
        return false
    end
    return true
end

function whatType(@nospecialize(dt))
    if <:(dt, Array)
        sub = whatType(eltype(dt))
        if sub == "diffe_dup"
            return "diffe_dup"
        elseif sub == "diffe_out"
            return "diffe_dup"
        else
            @assert(sub == "diffe_const")
            return "diffe_const"
        end
    end
    if <:(dt, Real)
        return "diffe_out"
    end
    if <:(dt, Int)
        return "diffe_const"
    end
    if <:(dt, String)
        return "diffe_const"
    end

    if ! hasfieldcount(dt)
        # just be safe for now
        return "diffe_dup"
    end
    @assert(hasfieldcount(dt))
    @assert(isstructtype(dt))
    passpointer = true
    if passpointer
        ty = "diffe_const"
        for (ft, fn) in zip(fieldtypes(dt), fieldnames(dt))
            sub = whatType(ft)
            if sub == "diffe_dup"
                ty = "diffe_dup"
            elseif sub == "diffe_out"
                ty = "diffe_dup"
            else
                @assert(sub == "diffe_const")
            end
        end
        return ty
    else
        ty = "diffe_const"
        for (ft, fn) in zip(fieldtypes(dt), fieldnames(dt))
            sub = whatType(ft)
            if sub == "diffe_dup"
                ty = "diffe_dup"
            elseif sub == "diffe_out"
                if ty != "diffe_dup"
                    ty = "diffe_out"
                end
            else
                @assert(sub == "diffe_const")
            end
        end
        return ty
    end
end

@generated function autodiff(f, args...)
    # Obtain the function and all it's dependencies in one handy module
    diffetypes = []
    autodifftypes = Type[f]
    i = 1
    while i <= length(args)
        push!(autodifftypes, args[i])
        dt = whatType(args[i])
        push!(diffetypes, dt)
        if dt == "diffe_dup"
            i+=1
        end
        i+=1
    end
    mod, ccf = irgen(Tuple{autodifftypes...}) 

    ctx = context(mod)
    rettype = convert(LLVMType, Float64)
    
    #argtypes2 = LLVMType[convert(LLVMType, T, true) for T in args]
    argtypes2 = LLVMType[]
    
    i = 1
    j = 1
    Base.println(typeof(ccf))
    Base.println(typeof(llvmtype(ccf)))
    Base.println(llvmtype(ccf))
    orig_params = parameters(ccf)
    for p in orig_params
        Base.println(llvmtype(p))
        push!(argtypes2, llvmtype(p))
        if diffetypes[i] == "diffe_dup"
            push!(argtypes2, llvmtype(p))
            i+=2
        else
            i+=1
        end
    end
    Base.println(argtypes2)
    

    # TODO get function type from ccf
    ft2  = LLVM.FunctionType(rettype, argtypes2)

    # create a wrapper Function that we will inline into the llvmcall
    # generated by in the end `call_function`
    llvmf = LLVM.Function(mod, "", ft2)
    push!(function_attributes(llvmf), EnumAttribute("alwaysinline", 0, ctx))
    linkage!(llvmf, LLVM.API.LLVMPrivateLinkage)

    # Create the FunctionType and funtion decleration for the intrinsic
    pt       = LLVM.PointerType(LLVM.Int8Type(ctx))
    ftd      = LLVM.FunctionType(rettype, LLVMType[pt], true)
    autodiff = LLVM.Function(mod, "llvm.autodiff.p0i8", ftd)

    params = LLVM.Value[]
    i = 1
    j = 1
    llvm_params = parameters(llvmf)
    while j < length(args)
        push!(params, MDString(diffetypes[i]))
        if diffetypes[i] == "diffe_dup"
            push!(params, llvm_params[j])
            j+=1
        end
        push!(params, llvm_params[j])
        j+=1
        i+=1
    end

    Builder(ctx) do builder
        entry = BasicBlock(llvmf, "entry", ctx)
        position!(builder, entry)

        tc = bitcast!(builder, ccf, pt)
        pushfirst!(params, tc)

        val = call!(builder, autodiff, params)

        #if T === Nothing
        #    ret!(builder)
        #else
            ret!(builder, val)
        #end
    end

    _mod[] = mod
    _args = (:(args[$i]) for i in 1:length(args))
    call_function(llvmf, Float64, Tuple{args...}, Expr(:tuple, _args...))
end

autodiff (generic function with 1 method)

In [ ]:
function sum(X::Array{Float64,1})
  acc = zero(eltype(X))
  @simd for x in X
      acc += x
  end
return acc
end
function sqcube(x::Array{Float64,1})
  sum(x .* x)
end

ar = [1.0, 2.0]
arp = [0.0, 0.0]
@show sqcube(ar)
@code_llvm optimize=false sqcube(ar)
#@show autodiff(sqcube, ar, arp)

In [ ]:
@generated function testgen(f, args...)
    # Obtain the function and all it's dependencies in one handy module
    diffetypes = []
    autodifftypes = Tuple{f, args[1]}
    mod, ccf = irgen(Tuple{autodifftypes...}) 
    return quote
    end
end

import Base: sum
function sum(X)
  acc = zero(eltype(X))
  @simd for x in X
      acc += x
  end
return acc
end

function sqcube(x::Array{Float64,1})
    sum(x .* x)
end

ar = [1.0, 2.0]
arp = [0.0, 0.0]
@show sqcube(ar)
@show testgen(sqcube, ar, arp)

In [48]:
whatType(Holder)

"diffe_dup"

In [3]:
mutable struct Holder
    bias::Float64
    name::String
end

Holder(val, name) = Holder(val, name)
import Base: zero
zero(model::Holder) = Holder(zero(model.bias), model.name)

function squareHolder(x::Holder)
    return x.bias*x.bias
end
inp = Holder(1.1,"hi")
outp = zero(inp)
autodiff(squareHolder, inp,outp)
outp

prefn:

define double @julia_squareHolder_1(%jl_value_t addrspace(10)* nonnull align 8 dereferenceable(16)) !dbg !22 {
top:
  %1 = addrspacecast %jl_value_t addrspace(10)* %0 to %jl_value_t addrspace(11)*, !dbg !25
  %2 = bitcast %jl_value_t addrspace(11)* %1 to double addrspace(11)*, !dbg !25
  %3 = load double, double addrspace(11)* %2, align 8, !dbg !25, !tbaa !29
  %4 = fmul double %3, %3, !dbg !31
  ret double %4, !dbg !28
}

saw metadata
prefn:

define internal {} @diffejulia_squareHolder_1(%jl_value_t addrspace(10)* nonnull align 8 dereferenceable(16), %jl_value_t addrspace(10)* %"'") !dbg !34 {
top:
  %"'ds4" = alloca %jl_value_t addrspace(11)*
  %"'ds3" = alloca double addrspace(11)*
  %"'de" = alloca double
  %_cache = alloca double
  %"'ds" = alloca double
  store double 0.000000e+00, double* %"'ds"
  store double 0.000000e+00, double* %"'de"
  store double addrspace(11)* null, double addrspace(11)** %"'ds3"
  store %jl_value_t addrspace(11)* null, %jl_value_t addrspace(11)*

Holder(2.2, "hi")

In [3]:
function jl_f2(f::Array{Float64,1})
    f[1] * f[1]
end

using InteractiveUtils

inp = Float64[1.0]
println("starting array diff")
@show jl_f2(inp)
res = Float64[0.0]
@show autodiff(jl_f2, inp, res)

starting array diff
jl_f2(inp) = 1.0
autodiff(jl_f2, inp, res) = 

prefn:

define double @julia_jl_f2_1(%jl_value_t addrspace(10)* nonnull align 16 dereferenceable(40)) !dbg !22 {
top:
  %1 = addrspacecast %jl_value_t addrspace(10)* %0 to %jl_value_t addrspace(11)*, !dbg !25
  %2 = bitcast %jl_value_t addrspace(11)* %1 to %jl_array_t addrspace(11)*, !dbg !25
  %3 = getelementptr inbounds %jl_array_t, %jl_array_t addrspace(11)* %2, i64 0, i32 1, !dbg !25
  %4 = load i64, i64 addrspace(11)* %3, align 8, !dbg !25, !tbaa !29
  %5 = icmp eq i64 %4, 0, !dbg !25
  br i1 %5, label %oob, label %idxend2, !dbg !25

oob:                                              ; preds = %top
  %6 = alloca i64, align 8, !dbg !25
  store i64 1, i64* %6, align 8, !dbg !25
  %7 = addrspacecast %jl_value_t addrspace(10)* %0 to %jl_value_t addrspace(12)*, !dbg !25
  call void @jl_bounds_error_ints(%jl_value_t addrspace(12)* %7, i64* nonnull %6, i64 1), !dbg !25
  unreachable, !dbg !25

idxend2:                                          ; preds = %top
  %8 = bitcast %jl_value_t addr

2.0


2.0

In [4]:
using LinearAlgebra

In [5]:
mutable struct LinearRegression
    # These values will be implicitly learned
    weights::Matrix{Float64}
    bias::Float64

    # These values will not be learned
    name::String
end

LinearRegression(nparams, name) = LinearRegression(randn(1, nparams), 0.0, name)
import Base: zero
zero(model::LinearRegression) = LinearRegression(zero(model.weights), zero(model.bias), model.name)

zero (generic function with 16 methods)

In [6]:
# Our linear prediction looks very familiar; w*X + b
function predict(model::LinearRegression, X)
    return model.weights * X .+ model.bias
end

# Our "loss" that must be minimized is the l2 norm  between our current
# prediction and our ground-truth Y
function loss(model::LinearRegression, X, Y)
    return norm(predict(model, X) .- Y, 2)
end

loss (generic function with 1 method)

In [7]:
weights_gt = [1.0, 2.7, 0.3, 1.2]'
bias_gt = 0.4

# Generate a dataset of many observations
X = randn(length(weights_gt), 10000)
Y = weights_gt * X .+ bias_gt

# Add a little bit of noise to `X` so that we do not have an exact solution,
# but must instead do a least-squares fit:
X .+= 0.001.*randn(size(X))

4×10000 Array{Float64,2}:
  0.94161    0.488997    0.583267  …  -0.626546  0.131863   0.041402 
  0.490982  -0.0528054  -1.38937       0.685289  0.108874  -0.902435 
  1.23898   -0.282981   -0.680838     -0.521935  0.242324  -0.0350945
 -1.37982    0.103517    0.835396      0.289123  0.385894  -0.731778 

In [9]:
model = LinearRegression(size(X, 1), "Example")


LinearRegression([1.054780994336981 0.34289714316689396 0.9134899621592786 0.3138791154561947], 0.0, "Example")

In [10]:
zero(model)

LinearRegression([0.0 0.0 0.0 0.0], 0.0, "Example")

In [52]:
mutable struct Holder
    bias::Float64
    name::String
end

Holder(val, name) = Holder(val, name)
import Base: zero
zero(model::Holder) = Holder(zero(model.bias), model.name)

function squareHolder(x::Holder)
    return x.bias*x.bias
end
inp = Holder(1.1,"hi")
outp = zero(inp)
autodiff(squareHolder, inp, outp)

2.2

In [11]:
function testmodel(m::LinearRegression)
    return loss(m, X[:,1], Y[1])
end

testmodel (generic function with 1 method)

In [15]:
<:

<: (built-in function)

In [16]:
x = fieldtypes(LinearRegression)[1]
println(x)
println(<:(x, Array))
println(whatType(x))

Array{Float64,2}
true
diffe_dup


In [62]:
whatType(Float64)

"diffe_out"

In [61]:
whatType(String)

"diffe_const"

In [73]:
typeof(model.weights)

Array{Float64,2}

In [70]:
fieldtypes(typeof(model))

(Array{T,2} where T, Float64, String)

In [17]:
whatType(LinearRegression)

"diffe_dup"

In [82]:
mutable struct Wrapper
    weights::Matrix
end
Wrapper(n) = Wrapper(randn(1,n))
wrap = Wrapper(3)
println(wrap)
println(fieldtypes(typeof(wrap)))
println(typeof(wrap.weights))

Wrapper([0.7498204602362356 -0.9834835117804321 -0.9033042994314854])
(Array{T,2} where T,)
Array{Float64,2}


In [18]:
println(<:(Array{T,2} where T,Array))
whatType(Array{T,2} where T)

true


"diffe_dup"

In [75]:
fieldtypes(typeof(model))

(Array{T,2} where T, Float64, String)

In [19]:
outp = zero(model)

LinearRegression([0.0 0.0 0.0 0.0], 0.0, "Example")

In [ ]:
autodiff(testmodel, model, outp)

In [14]:

# Calculate gradient upon `model` for the first example in our training set
@info("Calculating gradient (the first time can take a while to compile...)")
grads = Zygote.gradient(model) do m
    return loss(m, X[:,1], Y[1])
end

┌ Info: Calculating gradient (the first time can take a while to compile...)
└ @ Main In[14]:3


UndefVarError: UndefVarError: Zygote not defined